In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
true= pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
fake = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')

In [ ]:
true.head()

In [ ]:
fake.head()

In [ ]:
true['label']=1
fake['label']=0

df= pd.concat([true, fake], ignore_index= True)

In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
df.drop(['subject', 'date'], axis=1)

In [ ]:
df['text_processed']= df['text'].map(lambda x: re.sub('(Reuters)', "", x))
df['text_processed']= df['text_processed'].map(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))
df['text_processed']= df['text_processed'].map(lambda x: x.lower())


In [ ]:
df['text_processed']

In [ ]:
from nltk.stem import WordNetLemmatizer

def lemmatize(text):
    lm= WordNetLemmatizer()
    tokens= [lm.lemmatize(word) for word in text.split()]
    return " ".join(tokens)

In [ ]:
lemmatize(df['text_processed'][0])

In [ ]:
df['text_processed']= df['text_processed'].apply(lemmatize)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
y= df['label']
X= df.drop('label', axis=1)

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, stratify= y, random_state= 100)

In [ ]:
tfidf= TfidfVectorizer(stop_words= 'english', ngram_range= (1,3), lowercase= True, max_features= 5000)

X_train_transformed= tfidf.fit_transform(X_train['text_processed'])

X_test_transformed= tfidf.transform(X_test['text_processed'])

In [ ]:
X_train_transformed.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr= LogisticRegression()

lr.fit(X_train_transformed, y_train)

In [ ]:
y_pred= lr.predict(X_test_transformed)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)